# This code is created with the to clean weather station data collected on a Campbell Scientific Datalogger

## The goal of this code is to create new column names for better understanding and to drop erroneous data columns. 

In [1]:
# Import the packages needed to run this code.
import numpy as np
# Import pyplot for plotting
import matplotlib.pyplot as plt
#Import pandas for reading in and managing data
import pandas as pd
import math
import os
from datetime import date

In [2]:
# Set the working directory for where the AWS data is stored
os.chdir('/Users/f67f911/Desktop/Hourglass_WX/Data')

## First clean up the Hourly WX data

In [3]:
# Read in the data from the data file from the Campbell logger
# The data is stored in a .dat file, and we do not want the very first row because it contains unnecessary info
# The data is separated by commas, so we need to set the dilimeter
# The first two rows encompass the headers of our data, so we will combine those for the header
hg_hourly = pd.read_table('Raw/Hourglass_Hourly.dat',
                          skiprows = 1, delimiter = ',', header = [0,1])
# Join the first and second row values with a _ 
hg_hourly.columns = hg_hourly.columns.map('_'.join)
# Drop the columns with the descriptors of the the data retrieval
hg_hourly = hg_hourly.drop([0])
# Check the head of our data to make sure the data has been read in correctly
hg_hourly.head()

,TIMESTAMP_TS,RECORD_RN,AirTC_Deg C,AirTC_Avg_Deg C,RH_%,SnowDepth_Unnamed: 5_level_1,Q_Unnamed: 6_level_1,WS_ms_Max_meters/second,WS_ms_S_WVT_meters/second,WindDir_D1_WVT_Deg,...,LUp_W/m^2,LDn_W/m^2,CNR4TC_Deg C,CNR4TK_K,RsNet_W/m^2,RlNet_W/m^2,Albedo_%,Rn_W/m^2,LUpCo_W/m^2,LDnCo_W/m^2
1,2021-01-14 15:00:00,0.0,-9.99,-9.95,57.89,0.957,176,1.019,0.292,251.6,...,-65.6,-19.39,-10.89,262.3,2.254,-46.21,0.89,-43.96,202.6,248.8
2,2021-01-14 16:00:00,1.0,-11.38,-10.75,64.55,0.959,178,1.568,0.298,222.7,...,-59.29,-19.16,-12.63,260.5,0.451,-40.13,0.97,-39.68,201.9,242
3,2021-01-14 17:00:00,2.0,-11.74,-11.24,64.83,0.958,172,1.999,0.408,241.9,...,-59.3,-19.39,-12.88,260.3,-2.255,-39.91,3.5,-42.17,200.9,240.8
4,2021-01-14 18:00:00,3.0,-11.23,-11.68,61.18,0.953,172,2.47,0.559,283.5,...,-58.18,-18.94,-13.04,260.1,-2.481,-39.24,-1.75,-41.72,201.4,240.6
5,2021-01-14 19:00:00,4.0,-10.48,-11.26,57.2,0.951,180,2.156,0.594,269.2,...,-58.18,-19.62,-12.51,260.6,-2.932,-38.56,-2.25,-41.49,203.5,242


In [5]:
# Read in the column names to know which we need to keep and which we can remove and what names we want to change
hg_hourly.columns

Index(['TIMESTAMP_TS', 'RECORD_RN', 'AirTC_Deg C', 'AirTC_Avg_Deg C', 'RH_%',
       'SnowDepth_Unnamed: 5_level_1', 'Q_Unnamed: 6_level_1',
       'WS_ms_Max_meters/second', 'WS_ms_S_WVT_meters/second',
       'WindDir_D1_WVT_Deg', 'WindDir_SD1_WVT_Deg', 'T109_C_Avg_Deg C',
       'BP_mmHg_mmHg', 'SUp_W/m^2', 'SDn_W/m^2', 'LUp_W/m^2', 'LDn_W/m^2',
       'CNR4TC_Deg C', 'CNR4TK_K', 'RsNet_W/m^2', 'RlNet_W/m^2', 'Albedo_%',
       'Rn_W/m^2', 'LUpCo_W/m^2', 'LDnCo_W/m^2'],
      dtype='object')

In [6]:
# Drop the record column as it is unneccesary for future analysis and is just a count
hg_hourly = hg_hourly.drop(['RECORD_RN'], axis = 1)
# Check to make sure the data still looks correct
hg_hourly.head()

,TIMESTAMP_TS,AirTC_Deg C,AirTC_Avg_Deg C,RH_%,SnowDepth_Unnamed: 5_level_1,Q_Unnamed: 6_level_1,WS_ms_Max_meters/second,WS_ms_S_WVT_meters/second,WindDir_D1_WVT_Deg,WindDir_SD1_WVT_Deg,...,LUp_W/m^2,LDn_W/m^2,CNR4TC_Deg C,CNR4TK_K,RsNet_W/m^2,RlNet_W/m^2,Albedo_%,Rn_W/m^2,LUpCo_W/m^2,LDnCo_W/m^2
1,2021-01-14 15:00:00,-9.99,-9.95,57.89,0.957,176,1.019,0.292,251.6,68.82,...,-65.6,-19.39,-10.89,262.3,2.254,-46.21,0.89,-43.96,202.6,248.8
2,2021-01-14 16:00:00,-11.38,-10.75,64.55,0.959,178,1.568,0.298,222.7,50.27,...,-59.29,-19.16,-12.63,260.5,0.451,-40.13,0.97,-39.68,201.9,242
3,2021-01-14 17:00:00,-11.74,-11.24,64.83,0.958,172,1.999,0.408,241.9,45.79,...,-59.3,-19.39,-12.88,260.3,-2.255,-39.91,3.5,-42.17,200.9,240.8
4,2021-01-14 18:00:00,-11.23,-11.68,61.18,0.953,172,2.47,0.559,283.5,31.98,...,-58.18,-18.94,-13.04,260.1,-2.481,-39.24,-1.75,-41.72,201.4,240.6
5,2021-01-14 19:00:00,-10.48,-11.26,57.2,0.951,180,2.156,0.594,269.2,53.66,...,-58.18,-19.62,-12.51,260.6,-2.932,-38.56,-2.25,-41.49,203.5,242


In [7]:
# Clean up some of the columns to have more descriptive names
# Many of the names are not that descriptive, so we want to give them ones that make more sense
hg_hourly = hg_hourly.rename({
    'TIMESTAMP_TS':'Timestamp',
    'AirTC_Deg C':'AirTemp_DegC',
    'AirTC_Avg_Deg C':'Avg_AirTemp_DegC',
    'RH_%':'RH%',
    'SnowDepth_Unnamed: 5_level_1':'SnowDepth_m',
    'Q_Unnamed: 6_level_1':'Q',
    'WS_ms_Max_meters/second':'WindSpeed_Max_m/s',
    # I am not sure about this next variable, check and change to make more sense
    'WS_ms_S_WVT_meters/second':'WindSpeed_m/s',
    # Check the two below variables as well
    'WindDir_D1_WVT_Deg':'WindDir_D1_Deg',
    'WindDir_SD1_WVT_Deg':'WindDir_SD1_Deg',
    # Check the temperature variable below
    'T109_C_Avg_Deg C':'DataLogger_AvgTempC',
    'BP_mmHg_mmHg':'BarometricPressure_mmHg',
    'SUp_W/m^2': 'Incoming_Shortwave_W/m^2',
    'SDn_W/m^2': 'Outgoing_Shortwave_W/m^2',
    'LUp_W/m^2': 'Incoming_Longwave_W/m^2',
    'LDn_W/m^2': 'Outgoing_Longwave_W/m^2',
    # Check what the CNR4 is 
    'CNR4TC_Deg C':'CNR4TC_C',
    'CNR4TK_K': 'CNR4TK_K',
    # Check what the following variables mean: 'RsNet_W/m^2', 'RlNet_W/m^2'
    'Albedo_%':'Albedo%',
    # Check what the following variables mean: 'Rn_W/m^2', 'LUpCo_W/m^2', 'LDnCo_W/m^2' 
},
    axis = 1
)
# Check the dataframe to make sure the column names have been updated correctly
hg_hourly.head()

,Timestamp,AirTemp_DegC,Avg_AirTemp_DegC,RH%,SnowDepth_m,Q,WindSpeed_Max_m/s,WindSpeed_m/s,WindDir_D1_Deg,WindDir_SD1_Deg,...,Incoming_Longwave_W/m^2,Outgoing_Longwave_W/m^2,CNR4TC_C,CNR4TK_K,RsNet_W/m^2,RlNet_W/m^2,Albedo%,Rn_W/m^2,LUpCo_W/m^2,LDnCo_W/m^2
1,2021-01-14 15:00:00,-9.99,-9.95,57.89,0.957,176,1.019,0.292,251.6,68.82,...,-65.6,-19.39,-10.89,262.3,2.254,-46.21,0.89,-43.96,202.6,248.8
2,2021-01-14 16:00:00,-11.38,-10.75,64.55,0.959,178,1.568,0.298,222.7,50.27,...,-59.29,-19.16,-12.63,260.5,0.451,-40.13,0.97,-39.68,201.9,242
3,2021-01-14 17:00:00,-11.74,-11.24,64.83,0.958,172,1.999,0.408,241.9,45.79,...,-59.3,-19.39,-12.88,260.3,-2.255,-39.91,3.5,-42.17,200.9,240.8
4,2021-01-14 18:00:00,-11.23,-11.68,61.18,0.953,172,2.47,0.559,283.5,31.98,...,-58.18,-18.94,-13.04,260.1,-2.481,-39.24,-1.75,-41.72,201.4,240.6
5,2021-01-14 19:00:00,-10.48,-11.26,57.2,0.951,180,2.156,0.594,269.2,53.66,...,-58.18,-19.62,-12.51,260.6,-2.932,-38.56,-2.25,-41.49,203.5,242


In [8]:
# Clean up the timestamp data to get it into a useable format. This will allow us to filter by timestamp data in 
# future steps
hg_hourly['Timestamp'] =  pd.to_datetime((hg_hourly['Timestamp']))#,format='%m/%d/%Y %H:%M:%S')
# Check to make sure that TIMESTAMP data is now in a DateTime format
hg_hourly.head()

,Timestamp,AirTemp_DegC,Avg_AirTemp_DegC,RH%,SnowDepth_m,Q,WindSpeed_Max_m/s,WindSpeed_m/s,WindDir_D1_Deg,WindDir_SD1_Deg,...,Incoming_Longwave_W/m^2,Outgoing_Longwave_W/m^2,CNR4TC_C,CNR4TK_K,RsNet_W/m^2,RlNet_W/m^2,Albedo%,Rn_W/m^2,LUpCo_W/m^2,LDnCo_W/m^2
1,2021-01-14 15:00:00,-9.99,-9.95,57.89,0.957,176,1.019,0.292,251.6,68.82,...,-65.6,-19.39,-10.89,262.3,2.254,-46.21,0.89,-43.96,202.6,248.8
2,2021-01-14 16:00:00,-11.38,-10.75,64.55,0.959,178,1.568,0.298,222.7,50.27,...,-59.29,-19.16,-12.63,260.5,0.451,-40.13,0.97,-39.68,201.9,242
3,2021-01-14 17:00:00,-11.74,-11.24,64.83,0.958,172,1.999,0.408,241.9,45.79,...,-59.3,-19.39,-12.88,260.3,-2.255,-39.91,3.5,-42.17,200.9,240.8
4,2021-01-14 18:00:00,-11.23,-11.68,61.18,0.953,172,2.47,0.559,283.5,31.98,...,-58.18,-18.94,-13.04,260.1,-2.481,-39.24,-1.75,-41.72,201.4,240.6
5,2021-01-14 19:00:00,-10.48,-11.26,57.2,0.951,180,2.156,0.594,269.2,53.66,...,-58.18,-19.62,-12.51,260.6,-2.932,-38.56,-2.25,-41.49,203.5,242


In [9]:
# Set the name of the columns that we need to convert to float for future analysis
hourly_cols_toFloat = ['AirTemp_DegC','Avg_AirTemp_DegC','RH%','SnowDepth_m','WindSpeed_m/s','WindDir_D1_Deg','Incoming_Shortwave_W/m^2','Outgoing_Shortwave_W/m^2',
                'Incoming_Longwave_W/m^2','Outgoing_Longwave_W/m^2']
# Convert the data in these columns to type float
hg_hourly[hourly_cols_toFloat] = hg_hourly[hourly_cols_toFloat].apply(pd.to_numeric, errors='coerce', axis=1)
# Check that datatypes to make sure this updated correctly
hg_hourly.dtypes

Timestamp                   datetime64[ns]
AirTemp_DegC                       float64
Avg_AirTemp_DegC                   float64
RH%                                float64
SnowDepth_m                        float64
Q                                   object
WindSpeed_Max_m/s                   object
WindSpeed_m/s                      float64
WindDir_D1_Deg                     float64
WindDir_SD1_Deg                     object
DataLogger_AvgTempC                 object
BarometricPressure_mmHg             object
Incoming_Shortwave_W/m^2           float64
Outgoing_Shortwave_W/m^2           float64
Incoming_Longwave_W/m^2            float64
Outgoing_Longwave_W/m^2            float64
CNR4TC_C                            object
CNR4TK_K                            object
RsNet_W/m^2                         object
RlNet_W/m^2                         object
Albedo%                             object
Rn_W/m^2                            object
LUpCo_W/m^2                         object
LDnCo_W/m^2

In [10]:
# Now let's clean up some of the columns
# Make sure we don't have any incorrect values for relative humidity since RH can't be greater than 100
hg_hourly['RH%'] = np.where((hg_hourly['RH%'] > 100), np.nan, hg_hourly['RH%'])
# Set any value for negative snowdepth to NAN since we know negative SnowDepths can't exist
hg_hourly['SnowDepth_m'] = np.where((hg_hourly['SnowDepth_m'] < 0), np.nan, hg_hourly['SnowDepth_m'])
hg_hourly.head()

,Timestamp,AirTemp_DegC,Avg_AirTemp_DegC,RH%,SnowDepth_m,Q,WindSpeed_Max_m/s,WindSpeed_m/s,WindDir_D1_Deg,WindDir_SD1_Deg,...,Incoming_Longwave_W/m^2,Outgoing_Longwave_W/m^2,CNR4TC_C,CNR4TK_K,RsNet_W/m^2,RlNet_W/m^2,Albedo%,Rn_W/m^2,LUpCo_W/m^2,LDnCo_W/m^2
1,2021-01-14 15:00:00,-9.99,-9.95,57.89,0.957,176,1.019,0.292,251.6,68.82,...,-65.60,-19.39,-10.89,262.3,2.254,-46.21,0.89,-43.96,202.6,248.8
2,2021-01-14 16:00:00,-11.38,-10.75,64.55,0.959,178,1.568,0.298,222.7,50.27,...,-59.29,-19.16,-12.63,260.5,0.451,-40.13,0.97,-39.68,201.9,242
3,2021-01-14 17:00:00,-11.74,-11.24,64.83,0.958,172,1.999,0.408,241.9,45.79,...,-59.30,-19.39,-12.88,260.3,-2.255,-39.91,3.5,-42.17,200.9,240.8
4,2021-01-14 18:00:00,-11.23,-11.68,61.18,0.953,172,2.47,0.559,283.5,31.98,...,-58.18,-18.94,-13.04,260.1,-2.481,-39.24,-1.75,-41.72,201.4,240.6
5,2021-01-14 19:00:00,-10.48,-11.26,57.20,0.951,180,2.156,0.594,269.2,53.66,...,-58.18,-19.62,-12.51,260.6,-2.932,-38.56,-2.25,-41.49,203.5,242


In [11]:
# Create a column for longwave and shortwave albedo
# Create this based on the relationship between outgoing and incoming shortwave and longwave radiation
hg_hourly['Shortwave_Albedo_W/m^2'] = hg_hourly['Outgoing_Shortwave_W/m^2'].div(hg_hourly['Incoming_Shortwave_W/m^2']).astype(object)
hg_hourly['Longwave_Albedo_W/m^2'] = hg_hourly['Outgoing_Longwave_W/m^2'].div(hg_hourly['Incoming_Longwave_W/m^2']).astype(object)
# View the head of the data
hg_hourly.head()

,Timestamp,AirTemp_DegC,Avg_AirTemp_DegC,RH%,SnowDepth_m,Q,WindSpeed_Max_m/s,WindSpeed_m/s,WindDir_D1_Deg,WindDir_SD1_Deg,...,CNR4TC_C,CNR4TK_K,RsNet_W/m^2,RlNet_W/m^2,Albedo%,Rn_W/m^2,LUpCo_W/m^2,LDnCo_W/m^2,Shortwave_Albedo_W/m^2,Longwave_Albedo_W/m^2
1,2021-01-14 15:00:00,-9.99,-9.95,57.89,0.957,176,1.019,0.292,251.6,68.82,...,-10.89,262.3,2.254,-46.21,0.89,-43.96,202.6,248.8,0.890297,0.295579
2,2021-01-14 16:00:00,-11.38,-10.75,64.55,0.959,178,1.568,0.298,222.7,50.27,...,-12.63,260.5,0.451,-40.13,0.97,-39.68,201.9,242,0.969557,0.323157
3,2021-01-14 17:00:00,-11.74,-11.24,64.83,0.958,172,1.999,0.408,241.9,45.79,...,-12.88,260.3,-2.255,-39.91,3.5,-42.17,200.9,240.8,3.5,0.326981
4,2021-01-14 18:00:00,-11.23,-11.68,61.18,0.953,172,2.47,0.559,283.5,31.98,...,-13.04,260.1,-2.481,-39.24,-1.75,-41.72,201.4,240.6,-1.750554,0.325541
5,2021-01-14 19:00:00,-10.48,-11.26,57.20,0.951,180,2.156,0.594,269.2,53.66,...,-12.51,260.6,-2.932,-38.56,-2.25,-41.49,203.5,242,-2.250554,0.337229


In [12]:
# Now, clean up the shortwave and longwave albedo columns
# We know that since Albedo is a relationship, we can only have values between 0 and 1
# Therefore, any value that does not fit in this range, we need to set to NAN
hg_hourly['Shortwave_Albedo_W/m^2'] = np.where((hg_hourly['Shortwave_Albedo_W/m^2'] > 1) |(hg_hourly['Shortwave_Albedo_W/m^2'] < 0), 
                                               np.nan, hg_hourly['Shortwave_Albedo_W/m^2'])
hg_hourly['Longwave_Albedo_W/m^2'] = np.where((hg_hourly['Longwave_Albedo_W/m^2'] > 1) |(hg_hourly['Longwave_Albedo_W/m^2'] < 0),
                                              np.nan, hg_hourly['Longwave_Albedo_W/m^2'])
hg_hourly.head()

,Timestamp,AirTemp_DegC,Avg_AirTemp_DegC,RH%,SnowDepth_m,Q,WindSpeed_Max_m/s,WindSpeed_m/s,WindDir_D1_Deg,WindDir_SD1_Deg,...,CNR4TC_C,CNR4TK_K,RsNet_W/m^2,RlNet_W/m^2,Albedo%,Rn_W/m^2,LUpCo_W/m^2,LDnCo_W/m^2,Shortwave_Albedo_W/m^2,Longwave_Albedo_W/m^2
1,2021-01-14 15:00:00,-9.99,-9.95,57.89,0.957,176,1.019,0.292,251.6,68.82,...,-10.89,262.3,2.254,-46.21,0.89,-43.96,202.6,248.8,0.890297,0.295579
2,2021-01-14 16:00:00,-11.38,-10.75,64.55,0.959,178,1.568,0.298,222.7,50.27,...,-12.63,260.5,0.451,-40.13,0.97,-39.68,201.9,242,0.969557,0.323157
3,2021-01-14 17:00:00,-11.74,-11.24,64.83,0.958,172,1.999,0.408,241.9,45.79,...,-12.88,260.3,-2.255,-39.91,3.5,-42.17,200.9,240.8,NaN,0.326981
4,2021-01-14 18:00:00,-11.23,-11.68,61.18,0.953,172,2.47,0.559,283.5,31.98,...,-13.04,260.1,-2.481,-39.24,-1.75,-41.72,201.4,240.6,NaN,0.325541
5,2021-01-14 19:00:00,-10.48,-11.26,57.20,0.951,180,2.156,0.594,269.2,53.66,...,-12.51,260.6,-2.932,-38.56,-2.25,-41.49,203.5,242,NaN,0.337229


In [13]:
# Now, export the cleaned dataframe
# Exporting it as a .csv will help with further steps 
hg_hourly.to_csv('Cleaned/hg_hourly_cleaned.csv')

In [14]:
# For future analysis, it will be beneficial to export just the snow depth data from the weather station
hg_hourly_snowDepth = hg_hourly[['Timestamp','SnowDepth_m']]
hg_hourly_snowDepth.to_csv('Cleaned/hg_hourly_snowDepth.csv')

## Now let's do the same for the Daily Data

In [15]:
# Do the same for the daily data
hg_daily = pd.read_table('Raw/Hourglass_Daily.dat',
                          skiprows = 1, delimiter = ',', header = [0,1])
# Join the first and second row values with a _ 
hg_daily.columns = hg_daily.columns.map('_'.join)
# Drop the columns with the descriptors of the the data retrieval
hg_daily = hg_daily.drop([0])
# Check the head of our data to make sure the data has been read in correctly
hg_daily.tail()

,TIMESTAMP_TS,RECORD_RN,BattV_Min_Volts,PTemp_C_Deg C,WS_ms_S_WVT_meters/second,WindDir_D1_WVT_Deg,WindDir_SD1_WVT_Deg,T109_C_Avg_Deg C,T109_C_Max_Deg C,T109_C_TMx_Unnamed: 9_level_1,...,AirTC_Avg_Deg C,AirTC_Max_Deg C,AirTC_TMx_Unnamed: 21_level_1,AirTC_Min_Deg C,AirTC_TMn_Unnamed: 23_level_1,RH_Max_%,RH_TMx_Unnamed: 25_level_1,RH_Min_%,RH_TMn_Unnamed: 27_level_1,BP_mmHg_mmHg
896,2022-09-14 00:00:00,581.0,12.6,9.37,0.14,63.86,45.16,11.65,15.15,2022-09-13 17:14:15,...,NAN,NAN,2022-09-13 17:10:05,NAN,2022-09-13 17:10:05,93,2022-09-13 23:18:05,32.34,2022-09-13 17:42:25,703.9
897,2022-09-15 00:00:00,582.0,12.62,5.334,0.061,247.6,22.87,11.59,13.86,2022-09-14 16:57:50,...,8.7,14.64,2022-09-14 15:06:30,5.078,2022-09-14 23:06:40,96.7,2022-09-14 23:12:05,65.69,2022-09-14 15:04:15,706.2
898,2022-09-16 00:00:00,583.0,12.56,8.79,0.081,20.38,38.84,10.08,14.23,2022-09-15 16:38:10,...,NAN,NAN,2022-09-15 15:51:50,NAN,2022-09-15 15:51:50,97.9,2022-09-15 04:23:10,47.68,2022-09-15 16:03:55,706.2
899,2022-09-17 00:00:00,584.0,12.6,2.609,0.079,170,31.83,8.84,11.05,2022-09-16 00:00:45,...,5.734,9.1,2022-09-16 00:03:45,1.662,2022-09-16 23:44:10,97.6,2022-09-16 23:51:20,71.42,2022-09-16 00:03:30,704.6
900,2022-09-18 00:00:00,585.0,12.52,3.956,0.175,91.3,24.98,8.41,13.88,2022-09-17 15:51:05,...,NAN,NAN,2022-09-17 15:05:10,NAN,2022-09-17 15:05:10,98.3,2022-09-17 04:23:30,44.12,2022-09-17 16:46:50,703.4


In [16]:
# Now do the same for the daily data
hg_daily.columns

Index(['TIMESTAMP_TS', 'RECORD_RN', 'BattV_Min_Volts', 'PTemp_C_Deg C',
       'WS_ms_S_WVT_meters/second', 'WindDir_D1_WVT_Deg',
       'WindDir_SD1_WVT_Deg', 'T109_C_Avg_Deg C', 'T109_C_Max_Deg C',
       'T109_C_TMx_Unnamed: 9_level_1', 'T109_C_Min_Deg C',
       'T109_C_TMn_Unnamed: 11_level_1', 'SnowDepth_Max_Unnamed: 12_level_1',
       'SnowDepth_TMx_Unnamed: 13_level_1',
       'SnowDepth_Min_Unnamed: 14_level_1',
       'SnowDepth_TMn_Unnamed: 15_level_1', 'Q_Max_Unnamed: 16_level_1',
       'Q_Min_Unnamed: 17_level_1', 'Rain_mm_Tot_mm', 'AirTC_Avg_Deg C',
       'AirTC_Max_Deg C', 'AirTC_TMx_Unnamed: 21_level_1', 'AirTC_Min_Deg C',
       'AirTC_TMn_Unnamed: 23_level_1', 'RH_Max_%',
       'RH_TMx_Unnamed: 25_level_1', 'RH_Min_%', 'RH_TMn_Unnamed: 27_level_1',
       'BP_mmHg_mmHg'],
      dtype='object')

In [17]:
# Drop the record column as it is unneccesary for future analysis and is just a count
hg_daily = hg_daily.drop(['RECORD_RN'], axis = 1)
# Check to make sure the data still looks correct
hg_daily.head()

,TIMESTAMP_TS,BattV_Min_Volts,PTemp_C_Deg C,WS_ms_S_WVT_meters/second,WindDir_D1_WVT_Deg,WindDir_SD1_WVT_Deg,T109_C_Avg_Deg C,T109_C_Max_Deg C,T109_C_TMx_Unnamed: 9_level_1,T109_C_Min_Deg C,...,AirTC_Avg_Deg C,AirTC_Max_Deg C,AirTC_TMx_Unnamed: 21_level_1,AirTC_Min_Deg C,AirTC_TMn_Unnamed: 23_level_1,RH_Max_%,RH_TMx_Unnamed: 25_level_1,RH_Min_%,RH_TMn_Unnamed: 27_level_1,BP_mmHg_mmHg
1,2021-01-15 00:00:00,12.58,-8.96,1.365,177.7,93.6,-0.105,-0.094,2021-01-14 21:02:45,-0.115,...,-9.7,-6.876,2021-01-14 23:58:35,-12.43,2021-01-14 18:05:45,67.76,2021-01-14 16:25:50,40.94,2021-01-14 23:54:35,709
2,2021-01-16 00:00:00,12.51,-2.978,0.848,271,86.3,-0.107,-0.096,2021-01-15 06:30:50,-0.119,...,-3.141,2.249,2021-01-15 12:32:20,-9.79,2021-01-15 03:25:45,95.8,2021-01-15 21:45:35,38.18,2021-01-15 01:59:30,705.2
3,2021-01-17 00:00:00,12.57,-5.985,1.012,222.9,87.6,-0.116,-0.103,2021-01-16 06:29:10,-0.128,...,-5.91,-3.499,2021-01-16 00:00:05,-9.79,2021-01-16 18:18:50,94.4,2021-01-16 00:47:15,62.91,2021-01-16 22:03:05,705
4,2021-01-18 00:00:00,12.54,-6.62,1.408,232.1,86.3,-0.122,-0.113,2021-01-17 10:34:45,-0.134,...,-4.689,-2.335,2021-01-17 11:05:00,-7.313,2021-01-17 23:46:20,87.5,2021-01-17 14:54:00,55.92,2021-01-17 08:50:30,702.9
5,2021-01-19 00:00:00,12.49,-11.65,1.068,358.6,93.9,-0.124,-0.116,2021-01-18 23:41:20,-0.132,...,-8.25,-6.755,2021-01-18 15:13:15,-11.54,2021-01-18 23:47:10,92.3,2021-01-18 12:19:15,79.48,2021-01-18 00:02:05,709


In [18]:
# Clean up some of the columns to have more descriptive names
# Many of the names are not that descriptive, so we want to give them ones that make more sense
# This will change a bit from the Hourly data
hg_daily = hg_daily.rename({
    'TIMESTAMP_TS':'Timestamp',
    'WS_ms_S_WVT_meters/second':'WindSpeed_m/s',
    'WindDir_D1_WVT_Deg':'WindDir_Deg',
    'WindDir_SD1_WVT_Deg':'WindDir_SD1_Deg',
    'AirTC_Max_Deg C':'AirTemp_Max_C',
    'AirTC_TMx_Unnamed: 21_level_1':'AirTemp_Max_time',
    'AirTC_Min_Deg C':'AirTemp_Min_C',
    'AirTC_TMn_Unnamed: 23_level_1':'AirTemp_Min_time',
    # 'RH_%':'RH%',
    'SnowDepth_Max_Unnamed: 12_level_1':'SnowDepth_Max_m',
    'SnowDepth_TMx_Unnamed: 13_level_1':'SnowDepth_Max_time',
    'SnowDepth_Min_Unnamed: 14_level_1':'SnowDepth_Min_m',
    'SnowDepth_TMn_Unnamed: 15_level_1':'SnowDepth_Min_time',
    'RH_TMx_Unnamed: 25_level_1':'RH_Max_time',
    'RH_TMn_Unnamed: 27_level_1':'RH_Min_time', 
},
    axis = 1
)
# Check the dataframe to make sure the column names have been updated correctly
hg_daily.head()

,Timestamp,BattV_Min_Volts,PTemp_C_Deg C,WindSpeed_m/s,WindDir_Deg,WindDir_SD1_Deg,T109_C_Avg_Deg C,T109_C_Max_Deg C,T109_C_TMx_Unnamed: 9_level_1,T109_C_Min_Deg C,...,AirTC_Avg_Deg C,AirTemp_Max_C,AirTemp_Max_time,AirTemp_Min_C,AirTemp_Min_time,RH_Max_%,RH_Max_time,RH_Min_%,RH_Min_time,BP_mmHg_mmHg
1,2021-01-15 00:00:00,12.58,-8.96,1.365,177.7,93.6,-0.105,-0.094,2021-01-14 21:02:45,-0.115,...,-9.7,-6.876,2021-01-14 23:58:35,-12.43,2021-01-14 18:05:45,67.76,2021-01-14 16:25:50,40.94,2021-01-14 23:54:35,709
2,2021-01-16 00:00:00,12.51,-2.978,0.848,271,86.3,-0.107,-0.096,2021-01-15 06:30:50,-0.119,...,-3.141,2.249,2021-01-15 12:32:20,-9.79,2021-01-15 03:25:45,95.8,2021-01-15 21:45:35,38.18,2021-01-15 01:59:30,705.2
3,2021-01-17 00:00:00,12.57,-5.985,1.012,222.9,87.6,-0.116,-0.103,2021-01-16 06:29:10,-0.128,...,-5.91,-3.499,2021-01-16 00:00:05,-9.79,2021-01-16 18:18:50,94.4,2021-01-16 00:47:15,62.91,2021-01-16 22:03:05,705
4,2021-01-18 00:00:00,12.54,-6.62,1.408,232.1,86.3,-0.122,-0.113,2021-01-17 10:34:45,-0.134,...,-4.689,-2.335,2021-01-17 11:05:00,-7.313,2021-01-17 23:46:20,87.5,2021-01-17 14:54:00,55.92,2021-01-17 08:50:30,702.9
5,2021-01-19 00:00:00,12.49,-11.65,1.068,358.6,93.9,-0.124,-0.116,2021-01-18 23:41:20,-0.132,...,-8.25,-6.755,2021-01-18 15:13:15,-11.54,2021-01-18 23:47:10,92.3,2021-01-18 12:19:15,79.48,2021-01-18 00:02:05,709


In [19]:
# Clean up the timestamp data to get it into a useable format. This will allow us to filter by timestamp data in 
# future steps
hg_daily['Timestamp'] =  pd.to_datetime((hg_daily['Timestamp']))#,format='%m/%d/%Y %H:%M:%S')
hg_daily['SnowDepth_Max_time'] = pd.to_datetime(hg_daily['SnowDepth_Max_time']).dt.strftime('%Y-%m-%d-%H:%M:%S')
hg_daily['SnowDepth_Min_time'] = pd.to_datetime(hg_daily['SnowDepth_Min_time']).dt.strftime('%Y-%m-%d-%H:%M:%S')
hg_daily['AirTemp_Max_time'] = pd.to_datetime(hg_daily['AirTemp_Max_time']).dt.strftime('%Y-%m-%d-%H:%M:%S')
hg_daily['AirTemp_Min_time'] = pd.to_datetime(hg_daily['AirTemp_Min_time']).dt.strftime('%Y-%m-%d-%H:%M:%S')
hg_daily['RH_Max_time'] = pd.to_datetime(hg_daily['RH_Max_time']).dt.strftime('%Y-%m-%d-%H:%M:%S')
hg_daily['RH_Min_time'] = pd.to_datetime(hg_daily['RH_Min_time']).dt.strftime('%Y-%m-%d-%H:%M:%S')
# # Check to make sure that TIMESTAMP data is now in a DateTime format
hg_daily.head()

,Timestamp,BattV_Min_Volts,PTemp_C_Deg C,WindSpeed_m/s,WindDir_Deg,WindDir_SD1_Deg,T109_C_Avg_Deg C,T109_C_Max_Deg C,T109_C_TMx_Unnamed: 9_level_1,T109_C_Min_Deg C,...,AirTC_Avg_Deg C,AirTemp_Max_C,AirTemp_Max_time,AirTemp_Min_C,AirTemp_Min_time,RH_Max_%,RH_Max_time,RH_Min_%,RH_Min_time,BP_mmHg_mmHg
1,2021-01-15,12.58,-8.96,1.365,177.7,93.6,-0.105,-0.094,2021-01-14 21:02:45,-0.115,...,-9.7,-6.876,2021-01-14-23:58:35,-12.43,2021-01-14-18:05:45,67.76,2021-01-14-16:25:50,40.94,2021-01-14-23:54:35,709
2,2021-01-16,12.51,-2.978,0.848,271,86.3,-0.107,-0.096,2021-01-15 06:30:50,-0.119,...,-3.141,2.249,2021-01-15-12:32:20,-9.79,2021-01-15-03:25:45,95.8,2021-01-15-21:45:35,38.18,2021-01-15-01:59:30,705.2
3,2021-01-17,12.57,-5.985,1.012,222.9,87.6,-0.116,-0.103,2021-01-16 06:29:10,-0.128,...,-5.91,-3.499,2021-01-16-00:00:05,-9.79,2021-01-16-18:18:50,94.4,2021-01-16-00:47:15,62.91,2021-01-16-22:03:05,705
4,2021-01-18,12.54,-6.62,1.408,232.1,86.3,-0.122,-0.113,2021-01-17 10:34:45,-0.134,...,-4.689,-2.335,2021-01-17-11:05:00,-7.313,2021-01-17-23:46:20,87.5,2021-01-17-14:54:00,55.92,2021-01-17-08:50:30,702.9
5,2021-01-19,12.49,-11.65,1.068,358.6,93.9,-0.124,-0.116,2021-01-18 23:41:20,-0.132,...,-8.25,-6.755,2021-01-18-15:13:15,-11.54,2021-01-18-23:47:10,92.3,2021-01-18-12:19:15,79.48,2021-01-18-00:02:05,709


In [20]:
# Set the name of the columns that we need to convert to float for future analysis
daily_cols_toFloat = ['WindSpeed_m/s','WindDir_Deg', 'WindDir_SD1_Deg','SnowDepth_Max_m',
                      'SnowDepth_Min_m','AirTC_Avg_Deg C', 'AirTemp_Max_C',
                      'AirTemp_Min_C', 'RH_Max_%', 'RH_Min_%']

# Convert the data in these columns to type float
hg_daily[daily_cols_toFloat] = hg_daily[daily_cols_toFloat].apply(pd.to_numeric, errors='coerce', axis=1)
# Check that datatypes to make sure this updated correctly
hg_daily.dtypes

Timestamp                         datetime64[ns]
BattV_Min_Volts                           object
PTemp_C_Deg C                             object
WindSpeed_m/s                            float64
WindDir_Deg                              float64
WindDir_SD1_Deg                          float64
T109_C_Avg_Deg C                          object
T109_C_Max_Deg C                          object
T109_C_TMx_Unnamed: 9_level_1             object
T109_C_Min_Deg C                          object
T109_C_TMn_Unnamed: 11_level_1            object
SnowDepth_Max_m                          float64
SnowDepth_Max_time                        object
SnowDepth_Min_m                          float64
SnowDepth_Min_time                        object
Q_Max_Unnamed: 16_level_1                 object
Q_Min_Unnamed: 17_level_1                 object
Rain_mm_Tot_mm                            object
AirTC_Avg_Deg C                          float64
AirTemp_Max_C                            float64
AirTemp_Max_time    

In [21]:
# Now let's clean up some of the columns
# Make sure we don't have any incorrect values for relative humidity since RH can't be greater than 100
hg_daily['RH_Max_%'] = np.where((hg_daily['RH_Max_%'] > 100), np.nan, hg_daily['RH_Max_%'])
hg_daily['RH_Min_%'] = np.where((hg_daily['RH_Min_%'] > 100), np.nan, hg_daily['RH_Min_%'])
# Set any value for negative snowdepth to NAN since we know negative SnowDepths can't exist
hg_daily['SnowDepth_Max_m'] = np.where((hg_daily['SnowDepth_Max_m'] < 0), np.nan, hg_daily['SnowDepth_Max_m'])
hg_daily['SnowDepth_Min_m'] = np.where((hg_daily['SnowDepth_Min_m'] < 0), np.nan, hg_daily['SnowDepth_Min_m'])
hg_daily.head()

,Timestamp,BattV_Min_Volts,PTemp_C_Deg C,WindSpeed_m/s,WindDir_Deg,WindDir_SD1_Deg,T109_C_Avg_Deg C,T109_C_Max_Deg C,T109_C_TMx_Unnamed: 9_level_1,T109_C_Min_Deg C,...,AirTC_Avg_Deg C,AirTemp_Max_C,AirTemp_Max_time,AirTemp_Min_C,AirTemp_Min_time,RH_Max_%,RH_Max_time,RH_Min_%,RH_Min_time,BP_mmHg_mmHg
1,2021-01-15,12.58,-8.96,1.365,177.7,93.6,-0.105,-0.094,2021-01-14 21:02:45,-0.115,...,-9.700,-6.876,2021-01-14-23:58:35,-12.430,2021-01-14-18:05:45,67.76,2021-01-14-16:25:50,40.94,2021-01-14-23:54:35,709
2,2021-01-16,12.51,-2.978,0.848,271.0,86.3,-0.107,-0.096,2021-01-15 06:30:50,-0.119,...,-3.141,2.249,2021-01-15-12:32:20,-9.790,2021-01-15-03:25:45,95.80,2021-01-15-21:45:35,38.18,2021-01-15-01:59:30,705.2
3,2021-01-17,12.57,-5.985,1.012,222.9,87.6,-0.116,-0.103,2021-01-16 06:29:10,-0.128,...,-5.910,-3.499,2021-01-16-00:00:05,-9.790,2021-01-16-18:18:50,94.40,2021-01-16-00:47:15,62.91,2021-01-16-22:03:05,705
4,2021-01-18,12.54,-6.62,1.408,232.1,86.3,-0.122,-0.113,2021-01-17 10:34:45,-0.134,...,-4.689,-2.335,2021-01-17-11:05:00,-7.313,2021-01-17-23:46:20,87.50,2021-01-17-14:54:00,55.92,2021-01-17-08:50:30,702.9
5,2021-01-19,12.49,-11.65,1.068,358.6,93.9,-0.124,-0.116,2021-01-18 23:41:20,-0.132,...,-8.250,-6.755,2021-01-18-15:13:15,-11.540,2021-01-18-23:47:10,92.30,2021-01-18-12:19:15,79.48,2021-01-18-00:02:05,709


In [22]:
# Create a new column to hold the average daily snow depth variable
hg_daily['Average_SnowDepth_m'] = hg_daily.loc[:,['SnowDepth_Max_m','SnowDepth_Min_m']].mean(axis = 1)
hg_daily.head()

,Timestamp,BattV_Min_Volts,PTemp_C_Deg C,WindSpeed_m/s,WindDir_Deg,WindDir_SD1_Deg,T109_C_Avg_Deg C,T109_C_Max_Deg C,T109_C_TMx_Unnamed: 9_level_1,T109_C_Min_Deg C,...,AirTemp_Max_C,AirTemp_Max_time,AirTemp_Min_C,AirTemp_Min_time,RH_Max_%,RH_Max_time,RH_Min_%,RH_Min_time,BP_mmHg_mmHg,Average_SnowDepth_m
1,2021-01-15,12.58,-8.96,1.365,177.7,93.6,-0.105,-0.094,2021-01-14 21:02:45,-0.115,...,-6.876,2021-01-14-23:58:35,-12.430,2021-01-14-18:05:45,67.76,2021-01-14-16:25:50,40.94,2021-01-14-23:54:35,709,0.9495
2,2021-01-16,12.51,-2.978,0.848,271.0,86.3,-0.107,-0.096,2021-01-15 06:30:50,-0.119,...,2.249,2021-01-15-12:32:20,-9.790,2021-01-15-03:25:45,95.80,2021-01-15-21:45:35,38.18,2021-01-15-01:59:30,705.2,1.0010
3,2021-01-17,12.57,-5.985,1.012,222.9,87.6,-0.116,-0.103,2021-01-16 06:29:10,-0.128,...,-3.499,2021-01-16-00:00:05,-9.790,2021-01-16-18:18:50,94.40,2021-01-16-00:47:15,62.91,2021-01-16-22:03:05,705,1.0435
4,2021-01-18,12.54,-6.62,1.408,232.1,86.3,-0.122,-0.113,2021-01-17 10:34:45,-0.134,...,-2.335,2021-01-17-11:05:00,-7.313,2021-01-17-23:46:20,87.50,2021-01-17-14:54:00,55.92,2021-01-17-08:50:30,702.9,1.0330
5,2021-01-19,12.49,-11.65,1.068,358.6,93.9,-0.124,-0.116,2021-01-18 23:41:20,-0.132,...,-6.755,2021-01-18-15:13:15,-11.540,2021-01-18-23:47:10,92.30,2021-01-18-12:19:15,79.48,2021-01-18-00:02:05,709,1.0840


In [23]:
# Now, export the cleaned dataframe
# Exporting it as a .csv will help with further steps 
hg_daily.to_csv('Cleaned/hg_daily_cleaned.csv')

In [29]:
# Create a dataframe just containing the daily snow depth data
hg_daily_snowDepth = hg_daily[['Timestamp','SnowDepth_Max_m','SnowDepth_Min_m','Average_SnowDepth_m']]
hg_daily_snowDepth.head(n = 50)

,Timestamp,SnowDepth_Max_m,SnowDepth_Min_m,Average_SnowDepth_m
1,2021-01-15,0.974,0.925,0.9495
2,2021-01-16,1.074,0.928,1.0010
3,2021-01-17,1.064,1.023,1.0435
4,2021-01-18,1.067,0.999,1.0330
5,2021-01-19,1.146,1.022,1.0840
6,2021-01-20,1.139,1.093,1.1160
7,2021-01-21,1.113,1.079,1.0960
8,2021-01-22,2.345,1.067,1.7060
9,2021-01-23,1.150,1.063,1.1065
10,2021-01-24,2.343,1.065,1.7040


In [25]:
# Export to a daily snow depth .csv
hg_daily_snowDepth.to_csv('Cleaned/hg_daily_snowDepth.csv')